In [1]:
!pip install POT

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.0/823.0 kB 6.7 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import gdown
import gensim.downloader as api
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances, manhattan_distances
from scipy.spatial.distance import jaccard, dice, hamming
from scipy.stats import pearsonr, entropy
from sklearn.cluster import KMeans
from gensim.corpora import Dictionary
from gensim.similarities import WmdSimilarity
from gensim.models import KeyedVectors
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

url = "https://drive.google.com/uc?id=1y9qBtV71aY6OtEnhfQAxMTWXBBmdv63i"
output = "dataset.csv"
gdown.download(url, output, quiet=False, fuzzy=True)

Downloading...
From: https://drive.google.com/uc?id=1y9qBtV71aY6OtEnhfQAxMTWXBBmdv63i
To: /content/dataset.csv
100%|██████████| 93.0M/93.0M [00:03<00:00, 27.3MB/s]


'dataset.csv'

In [3]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [4]:
input_text = "At the age of 28, I, John Smith, am an enthusiastic and highly skilled data analyst seeking to advance my career by applying for a position in the data field. With over five years of experience in data analysis and a proven track record of successfully implementing data-driven strategies, I have honed my expertise in various analytical tools and methodologies. I hold a Bachelor's degree in Computer Science from the University of California, Berkeley, and a Master's degree in Data Science from Stanford University. Throughout my career, I have demonstrated exceptional analytical capabilities, strong problem-solving skills, and a deep understanding of statistical models and machine learning algorithms. My passion for uncovering insights from complex datasets and my commitment to continuous learning make me a valuable candidate for any data-driven organization."
input_text = input_text.lower()
input_text = re.sub(r'[^\w\s]', '', input_text)
tokens = word_tokenize(input_text)
stop_words = set(stopwords.words('english'))
tokens = [word for word in tokens if word not in stop_words]
lemmatizer = WordNetLemmatizer()
tokens = [lemmatizer.lemmatize(word) for word in tokens]
input_text = ' '.join(tokens)

In [5]:
glove_vectors = api.load("glove-wiki-gigaword-300")

[==================================================] 100.0% 376.1/376.1MB downloaded


In [6]:
def get_average_glove(tokens, model, num_features):
    valid_words = [word for word in tokens if word in model]
    if not valid_words:
        return np.zeros(num_features)
    return np.mean([model[word] for word in valid_words], axis=0)

dataset = pd.read_csv(output)

corpus_tokens = [word_tokenize(re.sub(r'[^\w\s]', '', doc.lower())) for doc in dataset["deskripsi_keterampilan_final"]]
corpus_tokens = [[lemmatizer.lemmatize(word) for word in tokens if word not in stop_words] for tokens in corpus_tokens]
num_features = 300
corpus_embeddings = np.array([get_average_glove(tokens, glove_vectors, num_features) for tokens in corpus_tokens])

input_tokens = word_tokenize(input_text)
input_embedding = get_average_glove(input_tokens, glove_vectors, num_features)

In [7]:
import pickle
with open('model_components.pkl', 'wb') as f:
    pickle.dump({
        'glove_vectors': glove_vectors,
        'corpus_embeddings': corpus_embeddings,
        'dataset': dataset
    }, f)

from google.colab import files
files.download('model_components.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [8]:
cosine_similarities = cosine_similarity([input_embedding], corpus_embeddings).flatten()
dataset["cosine_similarity"] = cosine_similarities

In [9]:
pd.set_option("display.max_colwidth",None)

In [10]:
top_similar = dataset.sort_values(by='cosine_similarity', ascending=False).head(20)
top_similar[["skill","job title","deskripsi_keterampilan_final","cosine_similarity"]]

skill  \
13111  Information Technology   
9203                    Other   
22397                Research   
19613                 Analyst   
29493               Education   
4277                  Science   
4486                  Science   
19252  Information Technology   
1784       Project Management   
19401             Engineering   
1142       Project Management   
5171                  Analyst   
6180               Consulting   
23306             Engineering   
7319              Engineering   
28867             Engineering   
23588                Research   
30445             Engineering   
4711         Customer Service   
30759  Information Technology   

                                                                       job title  \
13111           data science artificial intelligence machine learning internship   
9203   senior bioinformatics research scientist bioinformatic research scientist   
22397           group summer research program intern advanced concept technology   
19613                                                             data scientist   
29493                                    researcher doctor of philosophy student   
4277                                                       senior data scientist   
4486                                                       senior data scientist   
19252                                                             data scientist   
1784                                                              senior manager   
19401                                     research development civil engineering   
1142                                                       data scientist remote   
5171                                        senior assessment evaluation analyst   
6180                                       environmental engineer data scientist   
23306                                                          software engineer   
7319      director office of business units data science artificial intelligence   
28867                                             computer vision data scientist   
23588                          clinical data manager aggregate safety specialist   
30445                                                             data scientist   
4711                                                       senior data scientist   
30759                                                     machine learn engineer   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             